<a href="https://colab.research.google.com/github/s1c5000/Books/blob/main/handson/%EC%8B%AC%EC%B8%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D%ED%9B%88%EB%A0%A8%ED%95%98%EA%B8%B0_%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

11장에선 그레이디언트 소실과 폭주를 막기위해 여러 파라미터를 조정하는 방법을 설명한다  
초기화방법  
활성화함수  
배치정규화  
전이학습  
비지도 사전훈련  
옵티마이저  
학습률 스케줄링  
규제  
방법들을 설명하므로 매우 중요하다  

## 그레이디언트 소실과 폭주문제

역전파 알고리즘은 출력층에서 입력층으로 오차 그레이디언트를 전파하면서 진행된다

그런데 알고리즘이 하위층으로 진행될수록 그레이디언트가 점점 작아지는 경우가 많다. -> 그레이디언트 소실

반대로 그레이디언트가 점점 커져서 여러 층이 비정상적으로 큰 가중치로 갱신되며 발산한다. -> 그레이디언트 폭주  

불안정한 그레이디언트는 층마다 학습속도를 다르게해 훈련을 어렵게 만든다

시그모이드 활성화 함수와 당시 가장 인기있던 가중치 초기화 방법(정규분포) 때문에 그레이디언트 소실 많이 발생  
해당 방법은 각 층에서 출력의 분산이 입력의 분산보다 더 크다 -> 소실로 이어짐  
로지스틱이 0에서 멀어질 수록 기울기가 0에 수렴하기 때문에 그레이디언트는 점차 약해져 아래쪽층(입력과 가까운)에는 아무것도 도달하지 않게 된다.

### 글로럿과 He 초기화
적절한 신호가 흐르기 위해서는 각층의 출력에 대한 분산이 입력에 대한 분산과 같아야 한다  
-> 각 층의 연결 가중치를 무작위로 초기화 하는 것  
fan(avg) = (fan(in) + fan(out))/2  

글로럿 초기화를 사용하면 훈련 속도를 상당히 높일 수 있다.
이 초기화 전략을 세이비어초기화 or 글로럿 초기화 라고 한다.


ReLU활성화함수에 대한 초기화 전략을 논문저자의 이름을 따서 He초기화 라고 부른다.

케라스는 기본적으로 균등분포의 글로럿 초기화를 사용한다



### 수렴하지 않는 활성화 함수
활정화 함수를 잘못 선택하면 그레이디언트 소실이나 폭주로 이어 질 수 있다  
대부분 생물학적 뉴런과 비슷한 시그모이드 활성화 함수가 최선의 샌택일 것이라고 생각했다.  
하지만 다른 활성화 함수가 심층신경망에서 훨씬 더 잘 작동한다는 사실이 밝혀졌다.  
특히 ReLU함수는 특정 양숫값에 수렴하지 않는다는 큰 장점이 있다.(계산도 빠름)  
하지만 뉴런의 가중치가 바뀌어 훈련세트에 있는 모든 샘플에 대해 입력의 가중치 합이 음수가 되면 뉴런이 죽게된다. -> 죽은 ReLU  




LeakyReLU  
하이퍼파리미터 a가 이 함수가 새는(leaky)정도를 결정한다. 새는 정도란 z < 0 일때 이 함수의 기울기이고 보통 0.01로 설정한다. 이 작은 기울기가 LeakyLeLU를 절대 죽지 않게 만들어준다.

ELU  
다른 모든 ReLU변종들을 앞질렀고 훈련시간이줄고 테스트 세트 성능도 더 높았다.  
* z < 0일때 평균출력이 0에 가까워진다. 이는 그레이디언트 소실을 완화해준다.   
* z < 0이여도 그레이디언트가 0이 아니므로 죽은뉴런을 만들지않는다
* a=1이면 z = 0에서 급격히 변동하지 않으므로 z = 0을 포함해 모든 구간에서 매끄러워 경사하강법의 속도를 높여준다  

훈련동안에는 수렴속도가 빨라 느린 계산이 상쇄되지만 테스트 시에는 ELU를 사용한 네트워크가 ReLU를 사용한 네트워크보다 느릴 것아다

SELU  
스케일이 조정된 ELU활성화 함수의 변종  
완전연결층만 쌓아서 신경망을 만들고 모든 은닉층이 SELU활성화 함수를 사용한다면 네트워크가 자기 정규화된다.
훈련하는 동안 각 층의 출력이 평균 0과 표준편차 1을 유지하는 경향이 있다. 이는 그레이디언트 소실과 폭주를 막아준다.
SELU는 이런 종류의 네트워크(아주깊은 네트워크)에서 다른 활성화 함수보다 뛰어난 성능을 보여준다.

자기 정규화가 일어나기 위한 몇가지 조건이 있다.  
* 입력특성이 반드시 표준화(평균0, 표준편차1)되어야한다
* 모든 은닉층의 가중치는 르쿤 정규분포 초기화라 되어야한다. Kernel_initializer='lecun_normal'로설정
* 네트워크는 일렬로 쌓은 층으로 구성되어야한다. 순환 신경망이나 스킵연결과 같은 순차적이지 않은 구조에 SELU를 사용하면 자기 정규화되는 것이 보장되지않는다


심층신경망의 은닉층은 보통  
SELU > ELU > LeakkyReLU(그리고 변종들) > ReLU > tanh > 로지스틱  
속도가 가장 중요하다면 ReLU  
자기정규화 하지못하는 순환,스킵연결 구조라면 SELU는 부적합  

## 배치정규화
ELU 와 He 초기화로 그레이디언트 소실, 폭주를 크게 감소시킬 수 있지만 훈련동안 다시 발생 할 수도 있다.  

그레이디언트 문제를 막을 다른 방법으로 배치정규화가 있다  
각 층에서 활성화 함수를 통과하기 전이나 후에 모델에 연산을 하나 추가한다.  
입력을 원점에 맞추고 정규화한 다음 결과값의 스케일을 조정하고 이동시킨다   
입력데이터를 원점에 맞추고 정규화 하려면 알고리즘은 평균과 표준편차를 추정해야 한다. 이를 위해 현제 미니배치에서 입력의 평균과 표준편차를 평가한다.


배치 정규화를 적용한 모델은 이미지 분려 작업에 큰 성과를 냈다  
* 그레이디언트 소실문제가 크게 감소하여 하이퍼볼릭 탄젠트나 로지스틱 활성화 함수같은 수렴성을 가진 활성화 함수를 사용할 수 있다  
* 가중치 초기화에 네트워크가 훨씬 덜 민감해진다  
* 큰 학습률을 사용하여 학습 과정의 속도를 높일 수 있었다  
* 규제와 같은 역할

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False), # 활성화 함수 선언안함
    keras.layers.BatchNormalization(), #활성화 함수 전에 배치정규화를 한다.
    keras.layers.Activation("elu"), # 배치정규화 후 활성화 함수적용
    keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False), # use_bias로 편향 제외
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation="softmax")
])

## 전이학습
이미 학습해놓은 모델을 가져와서 사용할 수 있다.  
처음 몇번의 학습은 가중치를 동결하고 학습 해야한다.

## 옵티마이저


### 모맨텀 최적화
경사 하강법은 가중치에 대한 비용함수의 그레이디언트에 학습률을 곱한 것을 바로 차감하여 가중치를 갱신  
그레이디언트가 매우 작으면 매우 느려질 것이다

모멘텀 최적화는 이전 그레이디언트가 얼마였는지를 중요하게 생각한다  
모멘텀 최적화는 경사하강법보다 빠르게 평평한 지역을 탈출한다


In [ ]:
# modemtum = 0.9가 기본값
# 0(높은마찰저항) 1(마찰저항없음) 사이
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)

### 네스테로프 가속경사(NAG)
모맨텀 최적화의 변종으로 모맨텀 최적화보다 빠르다  
현재위치가 아니라 모맨텀의 방향으로 조금앞선 곳 에서 비용함수의 그래이디언트를 계산  
작은 개선들이 쌓여 NAG가 모맨텀보다 빠르다

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

### AdaGrad  
경사하강법은 전역 최적점 방향으로 곧장 향하지않고 가장 가파른 경사를 따라 빠르게 내려간뒤 느리게 이동한다
알고리즘이 이를 감지하고 전역 최적점쪽으로 더 정확한 방향을 잡으면 좋았을 것이다.  
AdaGrad알고리즘은 가장 가파른 차원을 따라 그레이디언트 벡터의 스케일을 감소시켜 이 문제를 해결한다

In [ ]:
# 너무 일찍 멈추는 경향이 있어 사용을 추천하진 않는다
optimizer = keras.optimizers.Adagrad(lr=0.001)

### RMSProp
AdaGrad는 너무 빨리 느려져서 전역 최적점에 수렴하지 못하는 위험이 있다.  
RMSProp알고리즘은 가장 최근 반복에서 비롯된 그레이디언트만 누적함으로써 이 문제를 해결

In [ ]:
# Adam이 나오기전까지 가장 선호되었음
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

### Adam과 Nadam최적화
적응적 모멘트 추정(adaptive moment estimation)을 의미하는 Adam은   
모멘텀 최적화와 RMSProp의 아이디어를 합친것이다.

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

Nadam 옵티마이저는 Adam 옵티마이저에 네스테로프 기법을 더한 것이다.  
종종 Adam보다 빠르게 수렴한다.

In [ ]:
optimizer = keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999)

## 학습률 스케줄링
큰 학습률로 시작하고 학습 속도가 느려질 때 학습률을 낮추면 최적의 고정 학습률보다 좋은 솔루션을 발견할 수 있다.  

* 거듭제곱 기반 스케줄링 : 학습률이 차차감소함  
* 지수가빈 스케줄링  :  s 스텝마다 10배씩 감소
* 구간별 고정 스케줄링  :  일정횟수동안 동일한 학습률 사용하고 점차 내린다  
* 성능기반 스케줄링  : 검증오차를 측정 후 오차가 줄어들지않으면 학습률을 감소  
* 1사이클 스케줄링  :  훈련 절반동안 초기학습률 lr0을 lr1까지 증가시킨다. 그 뒤 lr0까지 감소시킨다. 마지막 몇번의 에포크는 학습률을 소수점 몇째 자리까지 줄인다.  

여러 스케줄링 성능을 비교했을때 지수 기반 스케줄링이 선호된다

## 규제


### L2, L1 규제  
신경망의 연결 가중치를 제한하기 위해 L2규제를 사용  
많은 가중치가 0인 희소모델을 만들기위해 L1규제 사용  


In [ ]:
# 층의 연결 가중치에 규제강도 0.01을 사용해 L2규제를 적용한 모습
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

l2()함수는 훈련하는 동안 규제 손실을 계산하기 위해 각 스텝에서 호출되는 규제 객체를 반환한다. 이 손실은 최종손실에 합산된다.  
l1을 사용하고 싶으면 l1()함수를 사용하면 되고 둘다 사용하고 싶다면 l1_l2()를 사용하면 된다.

보통 네트워크의 모든 은닉층에 동일한 활성화함수, 초기화전력, 규제를 적용한다

### 드롭아웃
매 훈련스텝에서 각 뉴런은 드롭아웃될 확률p를 가진다.  
이번 스텝에서는 무시되지만 다음 스텝이서는 활성화 될 수 있다   
순환신경망에선 20 ~ 30%에 가깝고 합성곱 신경망에서는 40 ~ 50%에 가깝다.


입력값의 변화에 덜 민감해지고 결국 더 안정적인 네트워크가 되어 일반화 성능이 좋아진다  
10000개의 훈련 스텝을 진행하면 10000개의 다른 신경망을 훈련하게 된다.  
결과적으로 만들어진 신경망은 이 모든 신경망을 평균한 앙상블로 볼 수 있다.

## 요약
대부분의 경우 잘 맞는 설정들이다   
고정규칙으로 생각하면 안된다  

기본 DNN  

|하이퍼파라미터|기본값|
|------|---|
|커널초기화|He초기화
|활성화함수|ELU
|정규화|깊은 신경망이면 배치 정규화
|규제|조기종료(필요하면 L2)
|옵티마이져|모맨텀, RMSProp, Nadam
|학습률 스케줄| 1사이클

자기정규화를 위한 DNN  

|하이퍼파라미터|기본값|
|------|---|
|커널초기화|르쿤 초기화
|활성화함수|SELU
|정규화|없음(자기정규화)
|규제|필요하다면 알파 드롭아웃
|옵티마이져|모맨텀, RMSProp, Nadam
|학습률 스케줄| 1사이클